# Lecture 14: MCMC for the 2D Ising model

Last time we discussed Markov chain Monte Carlo (MCMC) simulations in general terms. If there are only a small number of configurations to consider, then this is straightforward. But what if the system is more complicated? 

As an example, we'll consider writing down a set of MCMC rules for the 2D Ising model. A small (say, $10\times10$) grid of spins already has many more configurations than are feasible to iterate over. 

Previously, we considered transition probabilities $T_{ij}$ between configurations labeled $i$ and $j$. The key relationship that the $T_{ij}$ need to satisfy is [detailed balance](https://en.wikipedia.org/wiki/Detailed_balance)

$$
P(i)T_{ij} = P(j)T_{ji}\,,
$$

where $P(i)$ is the probability of configuration $i$. Our first step toward practically implementing the MCMC simulation is to note that detailed balance is satisfied for a pair of configurations when $T_{ij}=T_{ji}=0$. In other words, not all transition probabilities need to be nonzero! However, we clearly can't set all of the $T_{ij}$ to zero. At the very least, we need to ensure that there is a "path" of nonzero transition probabilities between any two configurations of the system, so that we never get "stuck" on a subset of the configuration space. 

Practically, it's often useful to define *local* dynamics on the configuration space. For example, for the Ising model, we can set the probability for transitions between configurations that differ by one spin flip to be nonzero, and set the probability of all other transitions that involve multiple spin flips to be zero. This is useful because configurations that are very similar are likely to also have similar energies, so our transitions are unlikely to be rejected. Some advanced simulation algorithms do allow for more extensive changes in configurations, but we will not cover these during the course.

In the examples below, we'll use the [Metropolis rule](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) to set the transition probabilities for single spin flips,

$$
T_{ij} = \begin{cases} 
1 \qquad\qquad\,\, \text{if }E(j) < E(i)\\
e^{-\beta\left(E(j) - E(i)\right)} \;\; \text{if }E(j) > E(i)
\end{cases} \,.
$$


### Example: Laying out the 2D Ising model

Let's start by simply writing down an array that holds the configuration of an Ising spin system.

In [ ]:
import numpy as np
import numpy.random as rng

# Set the system size (N x N)
N = 10

# Define the configuration
s =  # FILL THIS IN

print(s)

### Steps of the MCMC algorithm

As mentioned above, we'll only choose nonzero transition probabilities for configurations that differ by one spin flip. In that case, the steps of our MCMC algorithm are as follows:

1. Select a spin
2. Compute the probability $P$ for that spin to flip
3. Draw a random number $r\in[0, 1)$ and flip if $r < P$

Let's run through the steps below.

### Step 1. Selecting a spin

How should we choose a spin to try to flip? A reasonable choice would be to pick the spin totally at random. We can do this by picking the corresponding lattice coordinates $(i, j)$ at random. Let's try this below.

In [ ]:
i =  # FILL THIS IN
j =  # FILL THIS IN

### Step 2. Computing the probability of a spin flip

Recall the Metropolis rule for computing the transition probability between two configurations labeled by $i$ and $j$,

$$
T_{ij} = \begin{cases} 
1 \qquad\qquad\,\, \text{if }E(j) < E(i)\\
e^{-\beta\left(E(j) - E(i)\right)} \;\; \text{if }E(j) > E(i)
\end{cases} \,.
$$

**Note: the $i$ and $j$ in this formula are not the same as the $i$ and $j$ lattice coordinates that we chose above!** Instead, here $i$ represents our starting configuration (without the spin flip), and $j$ represents the configuration with the spin flip. 

In this case, our energy function is

$$
E(\underline{\sigma}) = -\sum_{i=1}^{N}\sum_{j=1}^{N} J \left(\sigma_{i,j} \sigma_{i+1,j} + \sigma_{i,j}\sigma_{i,j+1}\right)\,.
$$

To make things a little bit simpler for us, we can choose *periodic boundary conditions* for our lattice, so that the $N$th spin is a nearest neighbor of the $1$st spin. In the equation above the index $N+1$ would thus correspond to $1$. Topologically, then, our 2D lattice is a torus.

**Exercise**: In the code block below, write an expression for the *change in energy* of the configuration `s` if the spin at site $(i, j)$ was flipped to have the opposite sign. Recall that the value of the spin at that site is given by `s[i, j]`.

In [ ]:
J = 0.4

delta_E =  # FILL THIS IN

print(delta_E)

Next we need to use `delta_E` to compute the probability of flipping the spin. According to the Metropolis rule, this is one if the energy decreases, else it is $\exp(-\beta \Delta E)$. Below we'll assume for simplicity that $T = 1/\beta = 1$.

In [ ]:
P = 1                     # this would be the probability if delta_E is negative
if delta_E>0:             # is delta_E positive?
    P = np.exp(-delta_E)  # if so, set the proper value for P
    
print('The probability of flipping spin (%d, %d) is %lf' % (i, j, P))

### Step 3. Test and flip (or not)

Now that we know the probability of a spin flip, we just need to generate a random number and use it to determine whether or not the spin will change value. For this we'll use `numpy`.

In [ ]:
r = rng.rand()    # choose a random number
if r<P:           # check: is r < P?
    s[i, j] *= -1 # if so, set s[i, j] to its negative (i.e., flip the spin!)

print('Got r = %lf, the value of spin (%d, %d) is now %d' % (r, i, j, s[i, j]))

### Reflect

Does the dynamics described above satisfy detailed balance? Is it possible to transition between all possible configurations of the system?

How would you scale up the steps above to run a long simulation of the Ising system?